In [1]:
import os
import sys
import copy
import json
import torch
import pickle
#import dotmap
import random
import logging
import numpy as np
from tqdm import tqdm
#from dotmap import DotMap
from itertools import chain
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from chairs import ChairsInContext

import dataset as shapeglot

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
dataset = 'shapeglot'
data_dir = './'
data_size = None
image_size = 64
override_vocab = None
context_condition = 'all'
split_mode = 'easy'

In [3]:
w2i = shapeglot.load_pkl_file('shapeglot/language/vocab_all.pkl')
i2w = shapeglot.invert_dict(w2i)

vocab = {'w2i' : w2i, 'i2w' : i2w}

torch.save(vocab, './models/shapeglot/vocab.pt')

In [4]:
print(vocab)

{'w2i': {'<NULL>': 0, '<START>': 1, '<END>': 2, '<UNK>': 3, '<DIA>': 4, '$': 5, '%': 6, "''": 7, "'d": 8, "'ll": 9, "'m": 10, "'s": 11, '(': 12, ')': 13, '+': 14, ',': 15, '-': 16, '--': 17, '...': 18, '....': 19, '.....': 20, '0': 21, '1': 22, '1.': 23, '1.40': 24, '10': 25, '100': 26, '12': 27, '15': 28, '16': 29, '2': 30, '2-3': 31, '2-d': 32, '2.': 33, '20': 34, '25': 35, '26': 36, '2d': 37, '2nd': 38, '2x4': 39, '3': 40, '3-d': 41, '3.': 42, '30': 43, '35': 44, '3d': 45, '3rd': 46, '4': 47, '40': 48, '45': 49, '4th': 50, '5': 51, '50': 52, '6': 53, '60': 54, '60s': 55, '7': 56, '70': 57, '70s': 58, '8': 59, '80': 60, '9': 61, '90': 62, '<': 63, '=d': 64, '>': 65, '?': 66, '@': 67, '[': 68, ']': 69, '^': 70, '_': 71, '``': 72, 'a': 73, 'a.': 74, 'abc': 75, 'able': 76, 'abnormal': 77, 'abnormally': 78, 'about': 79, 'above': 80, 'absolute': 81, 'absolutely': 82, 'abstract': 83, 'accent': 84, 'accents': 85, 'accident': 86, 'accommodates': 87, 'accomodate': 88, 'accordian': 89, 'accord

In [5]:
if dataset == 'chairs':
    DatasetClass = ChairsInContext
elif dataset == 'colors':
    DatasetClass = ColorsInContext
else:
    DatasetClass = ChairsInContext
data = DatasetClass('./'+str(dataset), image_size = 64, vocab = vocab, split = 'val', context_condition = 'far', train_frac = .95, val_frac = .05, image_transform = None)


Loading cleaned data from pickle.


In [6]:
all_imgs = None
all_labels = None
langs = None
count = 0
for i, (img, y, lang) in enumerate(data):
    print(i)
    label = [0, 0, 0]
    label[y] = 1
    if all_imgs is None:
        all_imgs = np.array([img.numpy()])
        all_labels = np.array([label])
        langs = np.array([lang.numpy()])
    else:
        all_imgs = np.append(all_imgs,np.array([img.numpy()]),0)
        all_labels = np.append(all_labels,np.array([label]),0)
        langs = np.append(langs,np.array([lang.numpy()]),0)
    seq = []
    if (i+1)%1000 == 0:
        print(langs)
        data_dict = {'imgs': all_imgs, 'labels': all_labels, 'langs': langs}
        np.savez_compressed('./data/'+str(dataset)+'/data_1000_'+str(count)+'.npz', **data_dict)
        count += 1
        all_imgs = None
        all_labels = None
        langs = None